In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from prepare import telco_work, telco_pipeline, rename_telco_proper

In [6]:
#import telco data and remove:
# payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'
df = telco_work()
#confirm data
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [3]:
#check total rows and columns
df.shape

(7043, 20)

In [7]:
#review data for nulls and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   object 
 3   dependents             7043 non-null   object 
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   object 
 6   multiple_lines         7043 non-null   object 
 7   online_security        7043 non-null   object 
 8   online_backup          7043 non-null   object 
 9   device_protection      7043 non-null   object 
 10  tech_support           7043 non-null   object 
 11  streaming_tv           7043 non-null   object 
 12  streaming_movies       7043 non-null   object 
 13  paperless_billing      7043 non-null   object 
 14  monthly_charges        7043 non-null   float64
 15  total_cha

Found 2 possible issues:

* Internet service type contains several nulls 
* Total charges should be a float but its currently an object

Lets first address the total charges. 

In [14]:
# Lets conveert the object to a float
df['total_charges'] = df['total_charges'].astype(float)

ValueError: could not convert string to float: ' '

We were unable to convert the object into a float

Received ValueError: could not convert string to float: ' '

We need to investigate why there are blanks in this column

In [16]:
# Lets use a boolean mask to discover 
df[df['total_charges'] == ' ']

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
945,Female,0,Yes,Yes,0,No,No phone service,Yes,Yes,Yes,Yes,Yes,No,No,56.05,,No,Two year,DSL,Credit card (automatic)
1731,Female,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.00,,No,Two year,NaN,Mailed check
1906,Male,0,No,Yes,0,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes,61.90,,No,Two year,DSL,Bank transfer (automatic)
2025,Male,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.70,,No,One year,NaN,Mailed check
2176,Male,0,No,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,,No,Two year,NaN,Mailed check
2250,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.35,,No,Two year,NaN,Mailed check
2855,Female,0,Yes,Yes,0,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,73.35,,No,Two year,DSL,Mailed check
3052,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.75,,No,Two year,NaN,Mailed check
3118,Female,0,Yes,Yes,0,No,No phone service,Yes,No,Yes,Yes,Yes,No,Yes,52.55,,No,Two year,DSL,Bank transfer (automatic)
4054,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,80.85,,No,Two year,DSL,Mailed check


We discovered that the reason for the blank values is due to new cutomers with no prior total charges. 

Lets replace the blanks with 0 and change the data type of the column

In [17]:
#conveert blanks to 0 and change type to float
df['total_charges'] = df['total_charges'].replace(' ', 0).astype(float)

In [21]:
#confirm data type has changed
print(df.dtypes)

gender                    object
senior_citizen             int64
partner                   object
dependents                object
tenure                     int64
phone_service             object
multiple_lines            object
online_security           object
online_backup             object
device_protection         object
tech_support              object
streaming_tv              object
streaming_movies          object
paperless_billing         object
monthly_charges          float64
total_charges            float64
churn                     object
contract_type             object
internet_service_type     object
payment_type              object
dtype: object


Lets now tackle the nulls in the internet_service_type column
first lets find out how many are there

In [22]:
#find any nulls in the data
df.isna().sum()

gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type    1526
payment_type                0
dtype: int64

Seems like internet sercice has 1500 nulls. 
lets investigate why

In [23]:
df[df.internet_service_type.isna()].head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
20,Female,1,Yes,No,50,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.20,1306.30,No,One year,NaN,Electronic check
23,Female,0,No,No,3,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,57.20,No,Month-to-month,NaN,Mailed check
24,Female,0,Yes,Yes,4,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,Month-to-month,NaN,Mailed check
27,Male,0,Yes,Yes,54,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.40,1090.60,No,Two year,NaN,Credit card (automatic)
28,Male,0,No,No,26,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.60,471.85,No,One year,NaN,Bank transfer (automatic)


It was null because the customer has no internet service.
Lets replace the null values with no internet service

In [24]:
df['internet_service_type'] = df['internet_service_type'].fillna("No internet service")

In [25]:
#confirm no nulls, and categorical options
print(df['internet_service_type'].isna().sum())
print(df['internet_service_type'].unique())

0
['DSL' 'Fiber optic' 'No internet service']


Lets check the unique values of evertying else to try and identify other possible issues

In [26]:
# First lets get the names of all the columsn
df.columns

Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'contract_type', 'internet_service_type', 'payment_type'],
      dtype='object')

In [38]:
# We assign the list of column names to a variable. Lets removed columns that hold numerical values. 
columns_to_check = ['gender', 'senior_citizen', 'partner', 'dependents',
                    'phone_service', 'multiple_lines', 'online_security', 'online_backup',
                    'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                    'paperless_billing', 'churn',
                    'contract_type', 'internet_service_type', 'payment_type']

# Calculate the maximum column name length for formatting
max_col_length = max(len(column) for column in columns_to_check)

# Loop through the columns and print unique values with aligned columns
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"{column.ljust(max_col_length)}  {unique_values}")

gender                 ['Female' 'Male']
senior_citizen         [0 1]
partner                ['Yes' 'No']
dependents             ['Yes' 'No']
phone_service          ['Yes' 'No']
multiple_lines         ['No' 'Yes' 'No phone service']
online_security        ['No' 'Yes' 'No internet service']
online_backup          ['Yes' 'No' 'No internet service']
device_protection      ['No' 'Yes' 'No internet service']
tech_support           ['Yes' 'No' 'No internet service']
streaming_tv           ['Yes' 'No' 'No internet service']
streaming_movies       ['No' 'Yes' 'No internet service']
paperless_billing      ['Yes' 'No']
churn                  ['No' 'Yes']
contract_type          ['One year' 'Month-to-month' 'Two year']
internet_service_type  ['DSL' 'Fiber optic' 'No internet service']
payment_type           ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


There are some categorical values that we do not need, but I will remove them after creating the dummy columns to save time

In [42]:
#create the dummies
df = pd.get_dummies(df)

In [43]:
#check the column names, identify dropable columns
df.columns

Index(['senior_citizen', 'tenure', 'monthly_charges', 'total_charges',
       'gender_Female', 'gender_Male', 'partner_No', 'partner_Yes',
       'dependents_No', 'dependents_Yes', 'phone_service_No',
       'phone_service_Yes', 'multiple_lines_No',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No', 'online_security_No internet service',
       'online_security_Yes', 'online_backup_No',
       'online_backup_No internet service', 'online_backup_Yes',
       'device_protection_No', 'device_protection_No internet service',
       'device_protection_Yes', 'tech_support_No',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No', 'streaming_tv_No internet service',
       'streaming_tv_Yes', 'streaming_movies_No',
       'streaming_movies_No internet service', 'streaming_movies_Yes',
       'paperless_billing_No', 'paperless_billing_Yes', 'churn_No',
       'churn_Yes', 'contract_type_Month-to-month', 'contract_type

In [44]:
# Drop unwanted columns
df = df.drop(
                        columns= ['gender_Female', 'partner_No', 'dependents_No', 'phone_service_No', 'multiple_lines_No', 
                                    'multiple_lines_No phone service', 'online_security_No', 'online_security_No internet service', 
                                    'online_backup_No', 'online_backup_No internet service', 'device_protection_No', 'device_protection_No internet service', 
                                    'tech_support_No', 'tech_support_No internet service', 'streaming_tv_No', 'streaming_tv_No internet service', 
                                    'streaming_movies_No', 'streaming_movies_No internet service', 'paperless_billing_No', 'churn_No', 
                                    'internet_service_type_No internet service'])

In [54]:
#check columns to confirm only wanted remains
pd.set_option('display.max_columns', None)
df.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_Yes,online_security_Yes,online_backup_Yes,device_protection_Yes,tech_support_Yes,streaming_tv_Yes,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,9,65.6,593.30,False,True,True,True,False,False,True,False,True,True,False,True,False,False,True,False,True,False,False,False,False,True
1,0,9,59.9,542.40,True,False,False,True,True,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True
2,0,4,73.9,280.85,True,False,False,True,False,False,False,True,False,False,False,True,True,True,False,False,False,True,False,False,True,False
3,1,13,98.0,1237.85,True,True,False,True,False,False,True,True,False,True,True,True,True,True,False,False,False,True,False,False,True,False
4,1,3,83.9,267.40,False,True,False,True,False,False,False,False,True,True,False,True,True,True,False,False,False,True,False,False,False,True


In [56]:
#lets rename our columns to something more legible

column_mapping = {
    'senior_citizen': 'senior_citizen',
    'tenure': 'tenure',
    'monthly_charges': 'monthly_charges',
    'total_charges': 'total_charges',
    'gender_Male': 'male',
    'partner_Yes': 'partner',
    'dependents_Yes': 'dependents',
    'phone_service_Yes': 'phone_service',
    'multiple_lines_Yes': 'multiple_lines',
    'online_security_Yes': 'online_security',
    'online_backup_Yes': 'online_backup',
    'device_protection_Yes': 'device_protection',
    'tech_support_Yes': 'tech_support',
    'streaming_tv_Yes': 'streaming_tv',
    'streaming_movies_Yes': 'streaming_movies',
    'paperless_billing_Yes': 'paperless_billing',
    'churn_Yes': 'churn',
    'contract_type_Month-to-month': 'contract_month',
    'contract_type_One year': 'contract_one_year',
    'contract_type_Two year': 'contract_two_year',
    'internet_service_type_DSL': 'internet_dsl',
    'internet_service_type_Fiber optic': 'internet_fiber_optic',
    'payment_type_Bank transfer (automatic)': 'payment_bank_transfer',
    'payment_type_Credit card (automatic)': 'payment_credit_card',
    'payment_type_Electronic check': 'payment_electronic_check',
    'payment_type_Mailed check': 'payment_mailed_check'
}

    # Rename the columns using the mapping
df.rename(columns=column_mapping, inplace=True)

# Print the new column names
print(df.columns)

Index(['senior_citizen', 'tenure', 'monthly_charges', 'total_charges', 'male',
       'partner', 'dependents', 'phone_service', 'multiple_lines',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn',
       'contract_month', 'contract_one_year', 'contract_two_year',
       'internet_dsl', 'internet_fiber_optic', 'payment_bank_transfer',
       'payment_credit_card', 'payment_electronic_check',
       'payment_mailed_check'],
      dtype='object')


In [57]:
df.head()

,senior_citizen,tenure,monthly_charges,total_charges,male,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,churn,contract_month,contract_one_year,contract_two_year,internet_dsl,internet_fiber_optic,payment_bank_transfer,payment_credit_card,payment_electronic_check,payment_mailed_check
0,0,9,65.6,593.30,False,True,True,True,False,False,True,False,True,True,False,True,False,False,True,False,True,False,False,False,False,True
1,0,9,59.9,542.40,True,False,False,True,True,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True
2,0,4,73.9,280.85,True,False,False,True,False,False,False,True,False,False,False,True,True,True,False,False,False,True,False,False,True,False
3,1,13,98.0,1237.85,True,True,False,True,False,False,True,True,False,True,True,True,True,True,False,False,False,True,False,False,True,False
4,1,3,83.9,267.40,False,True,False,True,False,False,False,False,True,True,False,True,True,True,False,False,False,True,False,False,False,True


In [58]:
#review data for nulls and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   senior_citizen            7043 non-null   int64  
 1   tenure                    7043 non-null   int64  
 2   monthly_charges           7043 non-null   float64
 3   total_charges             7043 non-null   float64
 4   male                      7043 non-null   bool   
 5   partner                   7043 non-null   bool   
 6   dependents                7043 non-null   bool   
 7   phone_service             7043 non-null   bool   
 8   multiple_lines            7043 non-null   bool   
 9   online_security           7043 non-null   bool   
 10  online_backup             7043 non-null   bool   
 11  device_protection         7043 non-null   bool   
 12  tech_support              7043 non-null   bool   
 13  streaming_tv              7043 non-null   bool   
 14  streaming_mov

In [59]:
train, val, test = telco_pipeline()

In [60]:
train.head()

,senior_citizen,tenure,monthly_charges,total_charges,male,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,churn,contract_month,contract_one_year,contract_two_year,internet_dsl,internet_fiber_optic,payment_bank_transfer,payment_credit_card,payment_electronic_check,payment_mailed_check
5609,0,14,76.45,1117.55,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False
2209,0,5,70.00,347.40,True,False,False,True,False,False,False,True,False,True,True,True,True,False,True,False,True,False,False,False,False,True
6919,0,35,75.20,2576.20,True,True,False,True,True,False,False,False,False,False,False,True,True,True,False,False,False,True,False,False,True,False
2284,0,58,86.10,4890.50,True,True,False,True,True,True,True,True,False,True,True,True,False,False,False,True,True,False,False,False,True,False
845,0,2,49.60,114.70,False,False,False,True,False,False,True,False,False,False,False,True,True,True,False,False,True,False,False,False,False,True


In [61]:
train.columns

Index(['senior_citizen', 'tenure', 'monthly_charges', 'total_charges', 'male',
       'partner', 'dependents', 'phone_service', 'multiple_lines',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn',
       'contract_month', 'contract_one_year', 'contract_two_year',
       'internet_dsl', 'internet_fiber_optic', 'payment_bank_transfer',
       'payment_credit_card', 'payment_electronic_check',
       'payment_mailed_check'],
      dtype='object')

In [4]:
train = rename_telco_proper(train)

In [8]:
train.head()

,Senior Citizen,Tenure,Monthly Charges,Total Charges,Male,Partner,Dependents,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Paperless Billing,Churn,Contract Month,Contract One Year,Contract Two Year,Internet DSL,Internet Fiber Optic,Payment Bank Transfer,Payment Credit Card,Payment Electronic Check,Payment Mailed Check
5609,0,14,76.45,1117.55,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False
2209,0,5,70.00,347.40,True,False,False,True,False,False,False,True,False,True,True,True,True,False,True,False,True,False,False,False,False,True
6919,0,35,75.20,2576.20,True,True,False,True,True,False,False,False,False,False,False,True,True,True,False,False,False,True,False,False,True,False
2284,0,58,86.10,4890.50,True,True,False,True,True,True,True,True,False,True,True,True,False,False,False,True,True,False,False,False,True,False
845,0,2,49.60,114.70,False,False,False,True,False,False,True,False,False,False,False,True,True,True,False,False,True,False,False,False,False,True


In [6]:
train.columns

Index(['Senior Citizen', 'Tenure', 'Monthly Charges', 'Total Charges', 'Male',
       'Partner', 'Dependents', 'Phone Service', 'Multiple Lines',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Paperless Billing', 'Churn',
       'Contract Month', 'Contract One Year', 'Contract Two Year',
       'Internet DSL', 'Internet Fiber Optic', 'Payment Bank Transfer',
       'Payment Credit Card', 'Payment Electronic Check',
       'Payment Mailed Check'],
      dtype='object')